<a href="https://colab.research.google.com/github/galeone/italian-machine-learning-course/blob/master/Introduzione_a_TensorFlow_2_0_e_TensorFlow_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x 
import tensorflow as tf
print(tf.__version__)

Google Colab ci fornisce un ambiente pronto all'uso per usare TensorFlow 2.0.
Nella cella precedente abbiamo usato il magic-command (comando non Python, ma specifico dei Jupyter Notebook) `%tensorflow_version` per impostare nel runtime corrente l'uso di TensorFlow 2.x.

TensorFlow 2.0 è eager by default: ogni riga di codice è eseguita in ordine sequenziale, esattamente così com'è scritta.

TensorFlow 1.x, invece, utilzzava un diverso paradigma di programazione. Infatti, TensorFlow era un framework che funzionava in modo *descrittivo*: prima veniva definita la computazione, e solo in seguito eseguita "all-at-once".

Usare TensorFlow 1.x all'interno di Jupyter Notebook era molto scomodo, ma fortunatamente la nuova versione permette di utilizzare ogni tool creato per Python (come i notebook), essendo TensorFlow 2.0 molto più "pythonico".

## Obiettivo

L'obietivo di questo notebook è risolvere un problema di classificazione su immagini, usando una rete completamente connesssa.

Gli step da seguire, propri di ogni buona pipeline di sviluppo di progetti Machine Learning sono:

- Ottenere ed Analizzare i dati
- Definire la pipeline di input
- Definire il modello
- Definire le metriche
- Definire il training loop
- Allenare il modello e misurare le metriche durante ed alla fine di ogni epoca
- Selezionare il modello migliore (basandosi sulla metrica di validation)
- Misurare le performance sullo split di test

Il tutto utilizzando come strumento di data visualization principale **TensorBoard**.

## Ottenere ed Analizzare i dati

In un caso d'uso reale, avere un dataset di dati etichettati è un processo lungo e noiso (che qualcuno però deve fare); fortunatamente, per sperimentare diversi algoritmi di machine learning, esistono dataset  (solitamente prodotti da universià o industrie) che sono diventati lo standard.

TensorFlow, ha deciso di standardizzare e semplificare il processo di ottenimento dei dataset mediante la libreria [**TensorFlow Datasets** (tfds)](https://www.tensorflow.org/datasets/).

Anziché dover manualmente scaricare e processare i dati, dai siti delle università/industrie, possiamo usare tfds, per (automaticamente):

- scaricare il dataset di dati grezzi
- applicare trasformazioni ai dati grezzi in modo tale da renderli usabili
- trasformare questi dati in `TFRecord` (formato ottimizzato per i dati usato da TensorFlow)
- ottenere un oggetto `tf.data.Dataset` (oggetti per pipeline di input altemete efficienti) pronto all'uso.

Essendo una libreria separata, è necessario scaricarla ed installarla nel sistema usando pip:


In [0]:
! pip install --upgrade tensorflow_datasets

Siamo ora pronti per conoscere TensorFlow datasets.

La libreria è molto semplice e pratica da usare: tutto si basa sul concetto di Dataset Builder. Un dataset builder è una classe (implementata all'interno di tfds) che contiene tutto il processo logico per scaricare, trasfromare, ed ottenere il dataset sotto forma di oggetto `tf.data.Dataset`.

Vedere la lista dei builders (e quindi dei dataset) disponibili è semplice:

In [0]:
import tensorflow_datasets as tfds
print(tfds.list_builders())
print(len(tfds.list_builders()))

I dataset disponibili sono ~100. Per ognuno di questi è disponibile una descrizione completa sul [catalogo online](https://www.tensorflow.org/datasets/catalog/overview).

Per sperimentare i nostri modelli di classificazione, scegliamo di utilizzare il dataset `"cifar10"`.

Questo dataset è un dataset tipicamente utilizzato per fare benchmark di algoritmi di computer vision ed è composto da immagini a colori 32x32.

TensorFlow datasets ci offre, mediante il metodo load, si ottenere sia il dataset che le **informazioni** relative al tipo di dati che questo contiene.

In [0]:
data, info = tfds.load("cifar10", with_info=True, split=tfds.Split.ALL)

In [0]:
print(info)

Grazie all'oggetto `DatasetInfo` abbiamo già una prima analisi del dataset:

Il dataset viene fornito direttamente con degli split:

- ci sono 50000 immagini di train
- ci sono 10000 immagini di test
- **non c'è un validation set** (dovremmo crearlo noi)
- le immagini sono `32 x 32 x 3` ed il loro tipo è `tf.uint8` (il ché implica valori in [0,255])
- le label sono 10 ed il tipo è `tf.int64` (uno scalare, non codificato one-hot)

Leggendo l'[API reference di TensorFlow Datasets](https://www.tensorflow.org/datasets/api_docs/python/tfds) è possibile trovare diverse funzioni messe a nostra disposizione per poter visualizzare ed analizzare il dataset.

Una delle più interessanti [`tfds.show_examples`](https://www.tensorflow.org/datasets/api_docs/python/tfds/show_examples) che dato un dataset e le sue informazioni, ci permette di visualizzare direttamente in un notebook (in quanto ritorna un oggetto matplotlib) alcuni samples dal dataset:


In [0]:
fig = tfds.show_examples(info, data, rows=4, cols=4)


Come visibile dall'immagine, all'interno del datataset è presente la coppia immagine label, ed all'interno dell'oggetto info, invece, è presente la relazione che lega la label testuale alla label numerica.

Dall'API documentation è possibile trovare i metodi [str2int](https://www.tensorflow.org/datasets/api_docs/python/tfds/features/ClassLabel#str2int) ed [int2str](https://www.tensorflow.org/datasets/api_docs/python/tfds/features/ClassLabel#int2str) che permettono di passare da stringa a label e viceversa.

Essendo TensorFlow 2.0 eager by default, possiamo iniziare ad utilizzarlo per creare un loop su oggetti di tipo `tf.Tensor`, prodotti all'operazione `tf.range` (equivalente alla `range` di Python). Nel loop visualizziamo la relazione tra label numerica e stringa:

In [0]:
for label in tf.range(10):
  print(label.numpy(), " -> ", info.features["label"].int2str(label))

## Dataset API: definire la pipeline di input

`data` è un oggetto di tipo `tf.data.Dataset`: la dataset API è **ottimizzata** per creare pipeline di input per il train di modelli.

L'API è basata sul **method chaining**: i metodi dell'oggetto dataset, applicano trasformazioni al dataset corrente, e ritornano un dataset con la trasofmrazione applicata.

La dataset API rappresenta correttamente il processo di ETL (Extract-Transform-Load) tipici di una pipeline di data science.

- TensorFlow Datasets è incaricato dell'estrazione dei dati e della prima trasformazione
- tf.data.Dataset con i suoi metodi applica la serie di trasformazioni atte a rendere utili i dati
- L'iterazione sull'oggetto dataset è il load dei dati in memoria

![etl](https://i.imgur.com/YRCqeAO.png)

La pipeline di trasformazioni che vogliamo applicare è questa:

- trasformare i dati da uint a float
- codificare one-hot le label
- scalare le immagini nel range [-1,1]
- "appiattire" (flatten) le immagini, per renderle anziché tensori `32 x 32 x 3`, dei tensori `32*32*3`
- Creare gli split di train, validation, test (tre oggetti `tf.data.Dataset`)
- Per ognuno di questi: creare dei batch di dimensione 32 per poter fare, successivamente, mini-batch gradient descent / valutazione in batch
- ottimizzare le performance della pipeline

In [0]:
def transform(row):
  # trasformare i dati da uint a float
  row["image"] = tf.image.convert_image_dtype(row["image"], dtype=tf.float32)
  # 1-hot
  row["label"] = tf.one_hot(row["label"], depth=10, on_value=1, off_value=0)
  # [-1,1] range
  row["image"] = (row["image"] - 0.5) * 2.
  # flatten
  row["image"] = tf.reshape(row["image"], (-1,))
  return row

dataset = data.map(transform)

# split, batch, prefetch
train = dataset.take(50000).batch(32).prefetch(1)
validation = dataset.skip(50000).take(5000).batch(32).prefetch(1)
test = dataset.skip(50000 + 5000).take(5000).batch(32).prefetch(1)

## Definizione del modello: Keras API

Keras è un API specification per la definizione ed il training di modelli di machine learning che TensorFlow ha deciso di adottare.

L'API è molto intuitiva da usare e si trova all'interno del modulo [`tf.keras`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/).

Keras offre tre differenti modi per creare un modello:

- Sequential API
- Functional API
- Subclassing

I layer offerti da Keras sono i più disparati: dai layer dense, ai layer di attivazione, ai layer di batch normalization, a quelli di convoluzioni per lavorare su immagini o pointcloud e molti altri.

Ogni layer è una classe da poter istanziare e configurare tramite i parametri del costruttore.

Per esempio, il costruttore del layer `tf.keras.layers.Dense` ha la seguente firma:

```python
__init__(
    units,
    activation=None,
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs
)
```

Nella quale possiamo specificare *ogni cosa* relativa all funzionamento del layer: dal numero di unità (neuroni), all'uso o meno del bias, al tipo di inizializzazione dei parametri.

Dato che la definizione di un modello è completamente arbitraria, possiamo provare a partire con un semplice modello (pochi neuroni per layer, per evitare l'overfitting), che riduce la dimensionalità dell'input layer per layer, fino ad arrivare a 10 neuroni di output (le classi).

Un semplice modello fully connected può essere visto come uno stack di layer `Dense`, ed è il caso d'uso canonico della Sequential API.

## Sequential API

In [0]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(256, activation=tf.nn.relu),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),
  tf.keras.layers.Dense(10) # Note the linear activation
])

Keras permette opzionalmente, di specificare la dimensione dell'input in fase di definizione del modello, oppure di lasciare che sia il Keras model alla prima esecuzione a determinarla in maniera automatica.

Questo può sembrare un particolare da poco, ma in realtà è qualcosa di fondamentale.

Conoscere a priori la dimensione dell'input permette di definire il **grafo computazionale** del modello completamente, e quindi poter visualizzare il "riassunto" completo del modello.

Ogni keras model offre il metodo `.summary()` per ottenere una visualizzazione tablellare della struttura del modello, ma per ottenre il numero dei parametri del primo layer, è **sempre** necessario conoscere la dimensione di input.

Infatti, per poter completamente definire la *matrice* dei pesi del primo layer, è necessario non solo il numero di neuroni, ma anche il numero di dimensioni dell'input.

Difatti, se proviamo a invocare il metodo `.summary()` sul modello appena creato, otteniamo il seguente errore:

In [0]:
model.summary()

Per poter visualizzare il summary completo, non avendo definito come attributo `input_shape` del primo layer, dobbiamo effettuare un **forward pass** del modello con un tensore di input (della dimensione coretta), in modo tale che Keras possa costrure il grafo computazionale.

In [0]:
fake_input = tf.zeros((1, 32*32*3))
out = model(fake_input)
model.summary()

Il numero di parametri è stato correttamente calcolato (e come è possibile vedere, per un modello così semplice siamo già oltre il milione di parametri), sebbene l'`output shape` risulti "multiple", anziché essere del valore corretto.

La **raccomadazione** è  di specificare **sempre** in fase di creazione del modello la dimensione dell'input, un modo tale da poter ottenere summary rappresentativi ed aiutare Keras nella definizione del modello stesso.

Possiamo quindi sovrascrivere il modelo precedente, creandolo ex-novo, ma specificando l'input shape. Per specificarla abbiamo due modi:

- O usare un `tf.keras.layers.Input` layer
- O usara il parametro del costruttore del primo layer dense `input_shape`

In [0]:
model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(32*32*3)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(256, activation=tf.nn.relu),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),
  tf.keras.layers.Dense(10) # Note the linear activation
])

In [0]:
model.summary()

In [0]:
tf.keras.utils.plot_model(model)

## Functional API

Un modo differente per definire i modelli, è quello di usare la functional API.

Ogni layer Keras è un oggetto **callable**: questo significa che è possibile utilizzare un oggetto istanziato come se fosse una funzione, che accetta un input e produce un output.

Per un modello con un singolo input ed un singolo output, totalmente sequenziale (come il nostro) non è necessario utlizzarla, i quanto Sequential soddisfa pienamente ogni requisito.

In ogni modo, essendo l'API più flessibile offerta da Keras per definire modelli, è bene conoscerla ed utlizzarla il più possibile, in modo tale da essere familiari con questa API quando si definiranno modelli con più input, più output e con relazioni tra i layer del modello.

In [0]:
inputs = tf.keras.layers.Input(shape=(32*32*3))
net = tf.keras.layers.Dense(512, activation=tf.nn.relu)(inputs)
net = tf.keras.layers.Dense(256, activation=tf.nn.relu)(net)
net = tf.keras.layers.Dense(128, activation=tf.nn.relu)(net)
out = tf.keras.layers.Dense(10)(net)
model = tf.keras.Model(inputs=inputs, outputs=out)
model.summary()

## Loss function

All'interno del modulo [`tf.keras.losses`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/losses) troviamo una lunga lista di loss function pronte all'uso.

Le loss disponibili sono tra le più disparate e scegliere quella corretta dipende da:

- il problema che stiamo risolvendo (classificazione, regressione, ...)
- il formato delle nostre label

Dato che:

- abbiamo codificato in one-hot le label
- **non** abbiamo applicato una non-linearità al layer di outut del modello (volutamente)

La nostra scelta deve ricadere sulle funzioni: **non** sparse e che accettano (unscaled) **logits** come input.

TensorFlow, per le loss, utilizza le keywords **sparse** e **logits** per indicare se la loss function accetta label scalari (non one-hot, ed applica la loss function stessa la conversione all'interno) e output di modelli **lineari**.

Quando la loss function accetta label scalari, allora è la loss function stessa che al suo interno applica la conversione a rappresentazione one-hot.

Quando la loss function accetta (o permette di specificare) `from_logits=True` significa che sarà la loss function stessa a applicare la non linearità corretta all'output del modello per il calcolo della loss.

Ad esempio, per un problema di classificazione multi-classe, con label rappresentate in one-hot, la loss function che viene utilzzata è la **categorical cross-entropy loss**.

Quello che vogliamo, è allenare la rete neurale per **produrre una probabilità su C classi** (10 in questo caso) data un immagine di input.

La loss calcola la **softmax activation** sull'output della rete (per riscalare i valori di output nel range probabilistico [0,1]) e dopo calcola la cross-entropy-loss.

**Softmax**

Softmax è una funzione di attivazione che riscala i valori di output di un classificatore nel range [0,1], in modo tale che la somma di tutti i valori predetti sia 1.

La softmax activation viene applicata agli **score** predetti dalla rete *s*; dato che gli elementi predetti rappresentano delle classi, questi score possono essere interpretati come probabilità (predizione aereoplano con probabilità 0.8, macchina con probabilità 0.1, ecc).

Per una data classe $s_i$, la funzione softmax viene calcolata come

$$ f(s)_i = \frac{e^{s_i}}{\sum_{j}^{C}{e^{s_j} }} $$

**Cross entropy loss**

La formula della categorical cross-entropy loss è data dall'applicazione della cross-entropy tra le label $t_i$ (one-hot) e le predizioni dopo il softmax.

$$ CE = - \sum_{i}^{C}{ t_i log(f(s)_i) } $$

Dato che la label sono codificate in one hot, solo il componente del vettore dove il valore è 1 concorre al calcolo della loss, mentre tutti gli altri valgono zero.

Dato il target vector (label reale), codificato in one hot $t$ e la sua componente ad uno nella posizione $p$ (quindi $t_p$), abbiamo che la formulazione della cross-entropy diventa:

$$ CE = -log\left(\frac{e^{s_p}}{\sum_{j}^{C}{e^{s_j}}}\right) $$

### Implementazione

Keras si occupa di realizzare **tutto** il calcolo della loss nella maniera più ottimizzata possibile e numericalmente stabile.

Come è facile notare, dato che è la loss stessa ad applicare la funzione di attivazione (**softmax**) all'output della rete, quando abbiamo definito il modello abbiamo evitato di aggiungerla all'output layer.

In [0]:
# Loss is a callable object
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

Abbiamo il modello `model`, abbiamo il dataset di train `train`, abbiamo la loss function da usare `loss`, ciò che rimane è la definizione del training loop, con annesse scelta dell'ottimizzatore e delle metriche da misurare.

## Training loop

TensorFlow 2.0 offre una maniera "avanzata" per definire ed implementare training loop.

Keras, d'altro canto, offre la sua maniera. In questo corso **non tratteremo** il modo Keras di definire ed eseguire i training loop, in quanto **nascondono troppi dettagli** e sono poco flessibili.

L'implementazione di un "custom training loop" è considerata avanzata, ma in realtà non è nulla di complesso. Anzi, avere controllo sulla fase di forward pass e di calcolo ed applicazione dei gradienti è utile per apprendere in maniera migliore il processo di training ed è senza dubbio l'opzione più flessibile.

### Metriche

Prima di definire il training loop, è bene scegliere che metriche misurare per tenere monitorate le performance ed identificare eventuali condizioni patologiche.

Dato che il dataset è bilanciato, possiamo misurare **l'accuracy**
Inoltre, possiamo misurare il valore della loss medio (sul batch) durante il training loop.

TensorFlow offre metriche per misurare il valore medio di un qualsiasi scalare che varia nel tempo (`tf.keras.metrics.Mean`), oppure per il calcolo del valore medio di una specifica metrica (e.g. `tf.keras.metrics.Accuracy`).

Ogni metrica implementa l'interfaccia standard di Keras relative alle metriche: questo ci garantisce che ogni oggeto appartenente al modulo `tf.keras.metrics` abbia i metodi:

- `update_state` (identico al metodo `__call__`): per computare la metrica
- `result` per ottenere il valore della metrica computato fin'ora
- `reset_state` per resettare lo stato della metrica al valore iniziale

In [0]:
accuracy = tf.keras.metrics.Accuracy()
mean_loss = tf.keras.metrics.Mean(name="loss")

## Loggare le metriche

Per loggare le metriche abbiamo due opzioni, da implementare **sempre** assieme quando si definisce una pipeline di ML (ben definita):

- loggare su stanard output/error
- loggare su **TensorBoard**

TensorBoard è un programma che viene installato assieme al modulo tensorflow e permette di visualizzare su grafici curve, istogrammi, dataset embedding, immagini, tracce audio, e molto altro.

È perfettamente integrato con i Jupyter notebook e tramite il magic command `%tensorboard` è possibile lansciare un'istanza del tensorboard server.

TensorBoard necessita di una cartella da "monitorare": all'interno di questa cartella vanno inseriti tutti i dati da loggare (summary).

TensorFlow, tramite il modulo `tf.summary` da la possibilità di salvare su file i valori delle metriche, delle immagin utilizzate e di ogni altro tipo di dato che il modulo `tf.summary` supporta.

Il concetto fondamentale per poter correttamente utilizzare i summary è quello di `FileWriter`.

Questo oggetto permette di creare un **contesto** e tramite questo, scrivere i dati all'interno della cartella monitorata da tensorboard.
Il **contesto** è fondamentale per una **buona organizzazione dei log**; infatti, è possibile creare un contesto di train, uno di validation ed uno di test, e visualizzare sullo stesso plot curve (in colori differenti) relative a contesti differenti.

Ad esempio, dato un `FileWriter` `writer`, possiamo definire un contesto tramite a keyword python `with` e scrivere all'interno del contesto creato dal writer, in questo modo:

```python
with writer.as_default():
  for step in range(100):
    # other model code would go here
    tf.summary.scalar("my_metric", 0.5, step=step)
    writer.flush()
```

Per il nostro caso, possiamo creare tre writer differenti

In [0]:
train_writer = tf.summary.create_file_writer("logs/train")
validation_writer = tf.summary.create_file_writer("logs/validation")
test_writer = tf.summary.create_file_writer("logs/test")

Avendo definito le metriche ed i file writer, siamo pronti a definire il training loop.

## Training loop

Il loop di training consiste in due parti. Durante ogni epoca di training, dobbiamo calcolare il valore della loss sul batch, **tenere traccia** delle operazioni effettuate durante il calcolo.

Tenere traccia delle operazioni fatte è di fondamentale importanza, in quanto possiamo utilizzare queste informazioni per calcolare **il gradiente**, quindi stimare la direzione dell'aggioranamento, ed usare un **ottimizzatore** per applicare l'update dei parametri nella direzione stimata.

TensorFlow 2.0 ci aiuta nella modularizzazione del codice: essendo eager by default, possiamo scrivere funzioni Python che effettuano determinate operazioni e mediante **tf.function** è possibile anche accelerare il calcolo di alcune di queste, convertendo il codice in una rappresentazione a grafo altamente ottimizzata.

In [0]:
# Define the optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)

def compute_loss(input_samples):
    predictions = model(input_samples["image"])
    loss_value = loss(input_samples["label"], predictions)
    return loss_value

@tf.function
def train_step(input_samples):
  with tf.GradientTape() as tape:
    loss_value = compute_loss(input_samples)

  gradient = tape.gradient(loss_value, model.trainable_variables)
  optimizer.apply_gradients(zip(gradient, model.trainable_variables))
  return loss_value

def measure_metrics(input_samples):
  predicted_labels = tf.argmax(model(input_samples["image"]), axis=1)
  accuracy.update_state(tf.argmax(input_samples["label"], axis=1), predicted_labels)
  mean_loss.update_state(compute_loss(input_samples))

Dopo aver definito i "macroblocchi" del nostro train, possiamo definire direttamente il training loop.

In [0]:
global_step = tf.Variable(0, dtype=tf.int64, trainable=False)
epoch_counter = tf.Variable(0, dtype=tf.int64, trainable=False)

def train_loop(num_epochs):
  
  for epoch in tf.range(epoch_counter, num_epochs):
    for input_samples in train:
      loss_value = train_step(input_samples)
      measure_metrics(input_samples)
      global_step.assign_add(1)

      if tf.equal(tf.math.mod(global_step, 100), 0):
        mean_loss_value = mean_loss.result() 
        accuracy_value = accuracy.result()
        mean_loss.reset_states()
        accuracy.reset_states()
        tf.print(f"[{global_step.numpy()}] loss value: ", mean_loss_value," - train acc: ", accuracy_value)
        with train_writer.as_default():
          tf.summary.scalar("loss", mean_loss_value, step=global_step)
          tf.summary.scalar("accuracy", accuracy_value, step=global_step)
          tf.summary.image("images", tf.reshape(input_samples["image"], (-1, 32,32,3)), step=global_step, max_outputs=5)
    # end of epoch: measure performance on validation set and log the values on tensorboard
    tf.print(f"Epoch {epoch.numpy() + 1 } completed")
    epoch_counter.assign(epoch + 1)
    # TODO: insert validation code here

Dopo aver definito la funzione di train, con annessa misura delle performance di train e validation, possiamo lanciare tensorboard e subito dopo invocare la funzione di train.

In [0]:
%load_ext tensorboard
%tensorboard --logdir logs

In [0]:
train_loop(num_epochs=2)

## Quali sono i problemi di questo training loop?

- Non stiamo misurando le performance di validation durante il training [**esercizio 1**]
- Al termine del train non stiamo misurando le performance sul test set [**esercizio 2**]
- Non c'è persistenza del modello: se il train deovesse interrompersi per qualsiasi motivo (fallimento hardware e simili) dovremmo re-iniziare il training dall'inizio, in quanto **non abbiamo salvato lo stato del modello**. [sezione successiva]
- C'è un problema con la visualizzazione delle immagini in TensorBoard: la funzione `tf.summary.image` si aspetta immagini scalate i [0,1] mentre le nostre immagini sono state riscalate in [-1,1] durante la `map_fn`. [**esercizio 3**]
- Non effettuiamo alcuna model selection: non gestendo la persistenza del modello e non misurando le performance sul validation set, non abbiamo tenuto monitorato l'overfitting (quando le performance di train sono troppo migliori delle performance di validation) [**esercizio 4**]

Alcuni di questi punti sono facilmente risolvibili con le conoscenze acquisite fin'ora, ma per gestire la persistenza è necessario introdurre il concetto di **training checkpoints**.

## Training Checkpoints

Salvare lo stato di un modello in TensorFlow 2.0 è davvero facile: tutto ciò che è necessario è craere un oggetto checkpoint ed assegnargli (direttamente nel costruttore) gli oggetti che vogliamo salvare.

Molti oggetti TensorFLow 2.0 sono "checkpointable", il ché significa che sono salvabili su disco dall'oggetto checkpoint (aka sono serializzabili).

Per usare un checkpoint è necessario un `CheckpointManager` che permette di gestirli.

esempio:

```python
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)
ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
  print("Restored from {}".format(manager.latest_checkpoint))
else:
  print("Initializing from scratch.")
```

Nel nostro esempio, vogliamo salvare lo stato del modello, dell'ottimizatore (anche se non stiamo usando un ottimizzatore che definisce variabili, ma è lo stesso una buona pratica) e il global step, così da poter riprendere il train dallo step esatto in cui è stato interrotto.

In [0]:
ckpt = tf.train.Checkpoint(step=global_step, optimizer=optimizer, model=model)
manager = tf.train.CheckpointManager(ckpt, 'ckpts', max_to_keep=3)
ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
  tf.print(f"Restored from {manager.latest_checkpoint}")
else:
  tf.print("Initializing from scratch.")

Avendo associato un CheckpointManager ad un oggetto `Checkpoint`, possiamo usarlo per salvare/ripristinare lo stato del modello.

Il manager altro non fa' che creare un'associazione tra l'oggetto `Checkpoint` **ed una cartella** (`ckpts`) dove verranno salvati gli oggetti "attaccati" al checkpoint.

Il metodo da utilizzare per salvare lo stato corrente è il metodo `.save()` del manager:

In [0]:
manager.save()
ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
  tf.print(f"Restored from {manager.latest_checkpoint}")

## Esercizio 1

Crea una nuova funzione `training_loop_v2` che aggiunga le misure delle performance sul validation set alla fine di ogni epoca. Usare il `FileWriter` corretto e gestire correttamente lo stato delle metriche.
Scrivere i risultati su tensorboard e verificare se una nuova curva appare sullo stesso grafico del training.


In [0]:
global_step = tf.Variable(0, dtype=tf.int64)

def train_loop(num_epochs):
  
  for epoch in tf.range(num_epochs):
    for input_samples in train:
      loss_value = train_step(input_samples)
      measure_metrics(input_samples)
      global_step.assign_add(1)

      if tf.equal(tf.math.mod(global_step, 100), 0):
        mean_loss_value = mean_loss.result() 
        accuracy_value = accuracy.result()
        mean_loss.reset_states()
        accuracy.reset_states()
        tf.print(f"[{global_step.numpy()}] loss value: ", mean_loss_value," - train acc: ", accuracy_value)
        with train_writer.as_default():
          tf.summary.scalar("loss", mean_loss_value, step=global_step)
          tf.summary.scalar("accuracy", accuracy_value, step=global_step)
          tf.summary.image("images", tf.reshape(input_samples["image"], (-1, 32,32,3)), step=global_step, max_outputs=5)
    # end of epoch: measure performance on validation set and log the values on tensorboard
    tf.print(f"Epoch {epoch.numpy() + 1 } completed")
    # TODO: insert validation code here

## Esercizio 2

Definire una funzione `test()` che misuri le performance sul test set ed invocarla al seguito dell'esecuzione della funzione `training_loop_v2` eseguita per 10 epoche.

In [0]:
def test():
  #TODO

training_loop_v2(num_epochs=10)
test()

## Esercizio 3

Creare una funzione che scali un tensore a valori in [0,1] in un tensore a valori in [-1,1] (codice già presente nel notebook).
Creare una seconda funzione che scali un tensore a valori in [-1,1] in [0,1]: usare `tf.summary.image` con immagini scalate nel range corretto ([0,1]): aggiornare ed eseguire le funzioni di training loop.

In [0]:
def rescale(image):
  return (image + 1.) / 2.


## Esercizio 4

La funzione `training_loop_v2` misura correttamente l'accuracy sul training set e sul validation set.

Modificare il codice della funzione per:

- ripristinare lo stato del modello dall'ultimo training step raggiungo prima di iniziare il training loop sulle nuove epoche richieste (usare un checkpoint ed un checkpoint manager per salvare il modello al termine di ogni epoca)
- dopo aver misurato l'accuracy di validation e l'accuracy di train (al termine di ogni epoca), usare un **diverso** checkpoint e checkpoint managert (su una diversa folder) per salvare il modello che ha raggiunto la miglior validation accuracy.
- Se per 2 epoche consecutive, le performance di train sono migliori delle performance di validation interrompere il training (early stopping basato sul confronto delle metriche)

## Esercizio 5

Creare un nuovo notebook per risolvere lo stesso problema, ma:

- non usare la funzione di one-hot encoding per le label, ma delegare l'encoding alla Keras loss adeguata: modificare ogni parte del codice necessaria ad usare le label scalari, anziché la rappresentazione one-hot.

## Esercizio 6

Sperimentare!

- Provare come variano le performance al variare del learning rate
- Cambiare dataset, scegliendo tra altri presenti in TensorFlow datasets (adeguare quindi il modello)
- Sperimentare come variano le performance al variare dell'ottimizzatore (vedesi lista degli ottimizzatori [nella documentazione](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/optimizers))
- Variare l'architettura del modello, cambiare numero di neuroni, numero di layer, funzioni di attivazione, struttura, ...

## Conclusione

La soluzione proposta per questo problema di classificazione di immagini è subottimale: stiamo usando una archietttura fully connected, con milioni di parametri, quando esiste una soluzione ben più efficiente, con un numero minore di parametri e con performance migliori: le reti neurali convoluzionali.

Questi tipo di rete ha rivoluzionato il campo della computer vision e del machine learning in generale: tutt'oggi sono i blocchi fondamentali per la stragrande maggioranza delle architetture che lavorano su immagini, audio (e anche per i modelli generativi!).

Nel prossimo notebook introdurremo l'operazione di convoluzione, le reti neurali convoluzionali, definiremo un'architettura deep usando stack di layer convoluzionali e risolveremo non solo il problema della classificazione, ma apprenderemo anche come **localizzare** un oggetto all'interno di una immagine, regredendo le coordinate della bounding box.